# Working With STAC

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/developmentseed/titiler/master?filepath=docs%2Fexamples%2FWorking_with_STAC_simple.ipynb)

### STAC: SpatioTemporal Asset Catalog

> The SpatioTemporal Asset Catalog (STAC) specification aims to standardize the way geospatial assets are exposed online and queried. A 'spatiotemporal asset' is any file that represents information about the earth captured in a certain space and time. The initial focus is primarily remotely-sensed imagery (from satellites, but also planes, drones, balloons, etc), but the core is designed to be extensible to SAR, full motion video, point clouds, hyperspectral, LiDAR and derived data like NDVI, Digital Elevation Models, mosaics, etc.

Ref: https://github.com/radiantearth/stac-spechttps://github.com/radiantearth/stac-spec

Using STAC makes data indexation and discovery really easy. In addition to the Collection/Item/Asset (data) specifications, data providers are also encouraged to follow a STAC API specification:  https://github.com/radiantearth/stac-api-spec

> The API is compliant with the OGC API - Features standard (formerly known as OGC Web Feature Service 3), in that it defines many of the endpoints that STAC uses. A STAC API should be compatible and usable with any OGC API - Features clients. The STAC API can be thought of as a specialized Features API to search STAC Catalogs, where the features returned are STAC Items, that have common properties, links to their assets and geometries that represent the footprints of the geospatial assets.


## Requirements

To be able to run this notebook you'll need the following requirements:
- ipyleaflet
- requests
- rasterio

`!pip install ipyleaflet requests rasterio`

In [1]:
import requests

from rasterio.features import bounds as featureBounds

from ipyleaflet import Map, basemaps, TileLayer, basemap_to_tiles, GeoJSON

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
titiler_endpoint = "https://api.cogeo.xyz/"  # Devseed temporary endpoint
stac_item = "https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_11055_6057_20070622/S5_11055_6057_20070622.json"

In [3]:
item = requests.get(stac_item).json()
print(item)

{'type': 'Feature', 'stac_version': '0.8.1', 'id': 'S5_11055_6057_20070622', 'properties': {'eo:gsd': 20, 'eo:bands': [{'name': 'Panchromatic', 'common_name': 'panchromatic', 'description': 'Panchromatic: 480-710 nm'}, {'name': 'Green', 'common_name': 'green', 'description': 'Green: 500-590 nm'}, {'name': 'Red', 'common_name': 'red', 'description': 'Red: 610-680 nm'}, {'name': 'Near Infrared', 'common_name': 'nir', 'description': 'Near Infrared: 780-890 nm'}, {'name': 'ShortWave Infrared', 'common_name': 'swir', 'description': 'ShortWave Infrared: 1580-1750 nm'}], 'proj:epsg': 3979, 'datetime': '2007-06-22T00:00:00Z'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-111.6453245, 60.81065789999892], [-111.2296762, 61.30928879999903], [-110.1583693, 61.093875199999], [-110.5877877, 60.59892389999882], [-111.6453245, 60.81065789999892]]]}, 'bbox': [-111.6453245, 60.59892389999882, -110.1583693, 61.30928879999903], 'links': [{'rel': 'collection', 'href': 'https://canada-spot-ortho.s3.am

In [4]:
print(list(item["assets"]))
for it, asset in item["assets"].items():
    print(asset["type"])

['pan', 'B1', 'B2', 'thumbnail', 'B3', 'B4']
image/tiff; application=geotiff; profile=cloud-optimized
image/tiff; application=geotiff; profile=cloud-optimized
image/tiff; application=geotiff; profile=cloud-optimized
image/jpeg
image/tiff; application=geotiff; profile=cloud-optimized
image/tiff; application=geotiff; profile=cloud-optimized


In [5]:
bounds = featureBounds(item)

m = Map(
    basemap=basemaps.OpenStreetMap.Mapnik,
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=8
)

geo_json = GeoJSON(data=item)
m.add_layer(geo_json)
m

Map(center=[60.95410634999892, -110.90184690000001], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [6]:
# Get Tile URL
r = requests.get(
    f"{titiler_endpoint}/stac/info",
    params = {
        "url": stac_item,
    }
).json()
print(r)

['pan', 'B1', 'B2', 'B3', 'B4']


### Display one asset

In [7]:
r = requests.get(
    f"{titiler_endpoint}/stac/tilejson.json",
    params = {
        "url": stac_item,
        "assets": "pan",
        "minzoom": 8,  # By default titiler will use 0
        "maxzoom": 14, # By default titiler will use 24
    }
).json()

m = Map(
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=10
)

tiles = TileLayer(
    url=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1
)
m.add_layer(tiles)
m

Map(center=[60.95410634999892, -110.90184690000001], controls=(ZoomControl(options=['position', 'zoom_in_text'…

### Merge Assets

In [8]:
# Get Tile URL
r = requests.get(
    f"{titiler_endpoint}/stac/tilejson.json",
    params = {
        "url": stac_item,
        "assets": "B3,B2,B1",  # Simple RGB combination
        "minzoom": 8,  # By default titiler will use 0
        "maxzoom": 14, # By default titiler will use 24
    }
).json()

m = Map(
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=10
)

tiles = TileLayer(
    url=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1
)
m.add_layer(tiles)
m

Map(center=[60.95410634999892, -110.90184690000001], controls=(ZoomControl(options=['position', 'zoom_in_text'…